In [18]:
import watvina.watvina_python as pywatvina
from watvina.rdkit2pdbqt import *

In [19]:
wv = pywatvina.WATVina(cpu=8,seed=0,verbosity=1)
#详细使用说明
#help(wv)

In [20]:
#Step 1: 从pdb文件读取受体
receptor_mol=Chem.MolFromPDBFile("rec.pdb", removeHs=False)
receptor_lines=MolToPDBQTBlock(receptor_mol, False, False, True)
wv.set_receptor_from_string(receptor_lines)

In [21]:
#Step 2: 从sdf文件中读取配体
ligand_mol=Chem.MolFromMolFile("i.sdf",removeHs=False)
ligand_lines=MolToPDBQTBlock(ligand_mol, True, False, True)
wv.set_ligand_from_string(ligand_lines)
wv.pose_atomids=[x-1 for x in wv.pose_atomids]
#wv.pose_atomids

In [22]:
# Step x:  读取水，或者药效团设置
#wv.set_pharmacophore_from_file("pharm.txt")
#wv.set_water("water.pdb",implicitsol=False)

In [23]:
# Step x: 设置打分权重和格点
#wv.set_watvina_weights(weight_vdw=0.184,weight_hb=1.00,weight_elep=0.25)
#wv.set_extra_constraints(weight_desol=-0.500, wclash_dist=0.500, weight_torsion=0.300)
wv.set_grid_dims(center_x=-0.50, center_y=30,center_z=16.4, size_x=19,size_y=23,size_z=23, granularity=0.375)

In [24]:
#Step 3: 根据打分方程预先计算不同距离的打分, 这个在global_search，score, optimize, relax时候都需要
#不带参数则只储存配体原子类型的格点能量
#wv.set_precalculate_sf()
#watvina的格点能量计算需要点时间, 所以我们如果针对不同的配体最好预先计算所有原子类型
wv.set_precalculate_sf()

In [25]:
#score only
wv.score()

-9.270598343046307

In [26]:
#local_only
wv.optimize()

Total energy (e_total)             : -12.544
Intramolecular Contribution         : 0.269
Desolvation (energy e_desolv)       :0.000
External_torsion                    :0.000
Affinity                            : -9.271 (kcal/mol)

Intermolecular contributions :
    VDW         : -47.413 : 0.184 = -8.724
    HBond       : -1.870 : 1.000 = -1.870
    Electrop    : -2.968 : 0.250 = -0.742
REMARK -47.413 -1.870 -2.968
e2[(inter+intra+desol+torsion)/tor]:-9.379
Performing local search ... done.
Total energy: -13.106
Pharmacophore Awards: 0.000
template ph4 match: 0.000
Torsion energy: 0.017
Desolvation_energy: 0.000
Intra energy: -1.382
Affinity: -9.725 (kcal/mol)
done.


-9.724713132928837

In [27]:
#Step 4: 计算原子类型的格点
#后面所有的配体都可以用这一个格点能量，不需要重复计算
wv.compute_watvina_maps(full_atomtypes=False)
wv.grid_score()

Computing WATVina grid ... done.


-11.288927151015656

In [28]:
wv.global_search(exhaustiveness=8,n_poses=5, min_rmsd=1.5,energy_range=3,population_size=8,
                 ga_searching=4,refinement=True, tramplitude=1.00)

Starting global_search
Using random seed: 474864159
Performing search ... Checking grid map...
Starting parallel search...
[==================================================] 100%done.
Refining results ... done.
+-----------+-------------+------------------------------------------------+--------+
| summary   |   RMSD_TO   |               Score contribution               | E_GRID |
+---+-------+------+------+-------+-------+-------+-------+-------+--------+--------+
|No.| score | best | init |  VDW  | HBond | Elect | Desol | Intra | Torsion|  TMPH4 |
+---+-------+------+------+-------+-------+-------+-------+-------+--------+--------+
!  1  -9.66   0.00   0.21  -48.80   -1.99   -3.33    0.00    0.25    0.09    0.00
!  2  -8.78   1.41   1.38  -46.22   -1.89   -3.01    0.00    0.34    0.41    0.00
!  3  -7.52   3.10   3.07  -44.49   -0.65   -3.27    0.00    0.27    0.30    0.00
!  4  -7.40   3.24   3.23  -43.19   -0.64   -2.74    0.00    0.18    0.25    0.00
!  5  -7.09   2.61   2.56  -3

In [29]:
print(f'Num of Docked Confs:{len(wv.poses_score)}')
print(f'Num of Atoms: {ligand_mol.GetNumAtoms()}')
print(f'Num of Atoms 2: {len(wv.poses_coords[0])}')
#wv.poses_coords[0]
#wv.pose_atomids

Num of Docked Confs:5
Num of Atoms: 58
Num of Atoms 2: 58


In [30]:
wv.poses_score
#wv.score()

[-9.659212497991078,
 -8.780574187034054,
 -7.5234509124516,
 -7.4021631127568766,
 -7.091518781263163]